In [1]:
import os
import base64
import exifread
import datetime
import polars as pl
import plotly.express as px
import pendulum
from tqdm import tqdm
from dash import Dash, dcc, html, Input, Output, State
from dash_extensions import EventListener

from jupyter_dash import JupyterDash
import webbrowser
import time

In [2]:
def encode_image(image_path):
    with open(image_path, "rb") as f:
        encoded = base64.b64encode(f.read()).decode()
        return f"data:image/png;base64,{encoded}"

In [3]:
# Helper function to convert GPS coordinates to decimal
def convert_to_decimal(coord, ref):
    degrees, minutes, seconds = coord
    decimal = degrees + (minutes / 60.0) + (seconds / 3600.0)
    if ref in ["S", "W"]:
        decimal = -decimal
    return decimal

In [4]:
# Extract GPS data using exifread
def extract_gps_data(image_path):
    try:
        with open(image_path, 'rb') as image_file:
            tags = exifread.process_file(image_file, stop_tag='GPS')

            timestamp = tags.get("Image DateTime")
            gps_latitude = tags.get("GPS GPSLatitude")
            gps_latitude_ref = tags.get("GPS GPSLatitudeRef")
            gps_longitude = tags.get("GPS GPSLongitude")
            gps_longitude_ref = tags.get("GPS GPSLongitudeRef")

            if gps_latitude and gps_latitude_ref and gps_longitude and gps_longitude_ref:
                lat = [float(x.num) / float(x.den) for x in gps_latitude.values]
                lon = [float(x.num) / float(x.den) for x in gps_longitude.values]

                lat_decimal = convert_to_decimal(lat, gps_latitude_ref.values)
                lon_decimal = convert_to_decimal(lon, gps_longitude_ref.values)

                return timestamp.values, lat_decimal, lon_decimal
        return timestamp.values, None, None
    except Exception as e:
        print(f"Error processing {image_path}: {e}")
        return None, None, None

In [5]:
# Process all images in a directory
def process_images(directory):
    data = []
    for filename in tqdm(os.listdir(directory)):
        if filename.lower().endswith(('.jpg', '.jpeg')):
            filepath = os.path.join(directory, filename)
            t, lat, lon = extract_gps_data(filepath)
            data.append({"filename": filename, "timestamp": t, "latitude": lat, "longitude": lon})
    return data

In [6]:
metadata = process_images('./photos/')

100%|█████████████████████████████████████| 4420/4420 [00:03<00:00, 1397.10it/s]


In [7]:
df = pl.DataFrame(metadata).with_columns(pl.col('timestamp').str.to_datetime('%Y:%m:%d %H:%M:%S')).filter(pl.col('latitude').is_not_null())
df.head()

filename,timestamp,latitude,longitude
str,datetime[μs],f64,f64
"""PXL_20241015_020821518.MP.jpg""",2024-10-14 22:08:21,40.73645,-73.986819
"""PXL_20240930_140806308.MP.jpg""",2024-09-30 10:08:06,40.712542,-73.993772
"""PXL_20241014_221031810.MP.jpg""",2024-10-14 18:10:31,40.753172,-73.978617
"""PXL_20241016_062749487.jpg""",2024-10-16 01:27:49,29.996244,-90.254758
"""PXL_20241017_155643771.jpg""",2024-10-17 10:56:43,29.960689,-90.057911


In [8]:
def datetime_display(slider_val, window_val):
    return f"{slider_val.strftime(DATETIME_FORMAT)} +{window_val}H"


def filter_df(df, min_val, time_window):
    return df.filter((pl.col('timestamp') >= min_val) & (pl.col('timestamp') <= min_val + datetime.timedelta(hours=time_window)))


def build_map(df):

    fig = px.density_mapbox(
        df.to_pandas(),
        lat="latitude",
        lon="longitude",
        hover_data={"timestamp": True},
        custom_data=["filename"],
        radius=10,
        center={"lat": 40.73, "lon": -73.97},
        mapbox_style="open-street-map",
        zoom=12,
    )

    fig.update_layout(margin={"l": 15, "r": 25, "t": 45, "b": 25}, coloraxis_showscale=False)
    fig.update_traces(hovertemplate="%{customdata[1]}")

    return fig

In [9]:
PHOTOS_DIR = './photos/'
REPLAY_SPEED = 500
DATETIME_FORMAT = '%A, %d %B %Y, %-H:%M'

min_date = df["timestamp"].min().replace(minute=0, second=0, microsecond=0)
max_date = df["timestamp"].max().replace(minute=0, second=0, microsecond=0) + datetime.timedelta(hours=1)

event_listener_js = {
    "event": "click",
    "props": ["clientX", "clientY"],
}

button_style = {"height": "20px"}

hover_image_style = {
    "position": "absolute",
    "top": "82px",
    "right": "50px",
    "width": "200px",
    "zIndex": "10",
    "border": "1px solid black",
    "display": "block",
}

click_image_style = {
    "position": "absolute",
    "top": "50%",
    "left": "50%",
    "transform": "translate(-50%, -50%)",
    "width": "600px",
    "zIndex": "10",
    "border": "1px solid black",
    "display": "block",
}


app = JupyterDash(__name__)

app.layout = html.Div([
    html.Div(
        id="datetime-display",
        style={
            "position": "absolute",
            "top": "10px",
            "left": "22px",
            "zIndex": "10",
        },
    ),
    html.Div([
        html.Button("Home", id="home-button", style=button_style),
        html.Button("Play", id="play-pause-button", style=button_style),
        html.Button("Backward", id="backward-button", style=button_style),
        html.Div(
            dcc.Slider(
                id="datetime-slider",
                min=min_date.timestamp(),
                max=max_date.timestamp(),
                step=3600,
                value=min_date.timestamp(),
                marks=None,
                updatemode="drag",
            ),
            style={"margin-top": "3px"},
        ),
        html.Button("Forward", id="forward-button", style=button_style),
        html.Button("+", id="plus-button", style=button_style),
        html.Button("-", id="minus-button", style=button_style),

    ],
        style={
            "display": "grid",
            "grid-template-columns": "8% 8% 8% 52% 8% 8% 8%",
            "position": "absolute",
            "top": "30px",
            "left": "22px",
            "width": "50%",
            "height": "20px",
            "zIndex": "10",
        },
    ),
    EventListener(
        dcc.Graph(id="figure", figure=build_map(df), style={"height": "100vh", "width": "100vw"}),
        events=[event_listener_js],
        id="figure-clicks",
    ),
    EventListener(
        html.Img(id="click-image", style={"display": "block"}),
        events=[event_listener_js],
        id="click-image-clicks",
    ),
    html.Img(id="hover-image"),
    dcc.Interval(id="datetime-ticker", interval=REPLAY_SPEED, n_intervals=0, disabled=True),
    dcc.Store(id="click-image-visible", data=False),
    dcc.Store(id="datetime-window", data=1),
    dcc.Store(id="home-button-flag", data=True),
])

In [10]:
@app.callback(
    Output("hover-image", "src"),
    Output("hover-image", "style"),

    Input("figure", "hoverData"),
    Input("click-image-visible", "data")
)
def display_hover_image(hoverData, click_image_visible):
    if hoverData and not click_image_visible:
        image_url = os.path.join(PHOTOS_DIR, hoverData["points"][0]["customdata"][0])
        return encode_image(image_url), hover_image_style
    return "", {"display": "none"}


@app.callback(
    Output("click-image", "src"),
    Output("click-image", "style"),
    Output("figure", "clickData"),
    Output("figure", "hoverData"),
    Output("click-image-visible", "data"),

    Input("figure", "clickData"),
    Input("figure-clicks", "event"),
    Input("click-image-clicks", "event"),
)
def toggle_click_image(clickData, figure_clicks, click_image_clicks):
    if clickData:
        image_url = os.path.join(PHOTOS_DIR, clickData["points"][0]["customdata"][0])
        return encode_image(image_url), click_image_style, None, None, True
    return "", {"display": "none"}, None, None, False


@app.callback(
    Output("play-pause-button", "children", allow_duplicate=True),
    Output("datetime-display", "children", allow_duplicate=True),
    Output("datetime-ticker", "disabled", allow_duplicate=True),
    Output("datetime-slider", "disabled", allow_duplicate=True),
    Output("datetime-slider", "value", allow_duplicate=True),
    Output("datetime-window", "data", allow_duplicate=True),
    Output("figure", "figure", allow_duplicate=True),
    Output("home-button-flag", "data", allow_duplicate=True),

    Input("home-button", "n_clicks"),

    prevent_initial_call="initial_duplicate",
)
def home_button(n_clicks):
    print(f"{min_date.strftime(DATETIME_FORMAT)} +1H")
    return "Play", f"{min_date.strftime(DATETIME_FORMAT)} +1H", True, False, min_date.timestamp(), 1, build_map(df), True


@app.callback(
    Output("home-button-flag", "data"),

    Input("datetime-slider", "value"),
)
def reset_home_button_flag(slider_val):
    return False


@app.callback(
    Output("play-pause-button", "children", allow_duplicate=True),
    Output("datetime-ticker", "disabled", allow_duplicate=True),
    Output("datetime-slider", "disabled", allow_duplicate=True),

    Input("play-pause-button", "n_clicks"),
    State("datetime-slider", "disabled"),

    prevent_initial_call=True,
)
def play_pause_button(n_clicks, not_playing):
    if not_playing:
        return "Play", True, False
    return "Pause", False, True


@app.callback(
    Output("datetime-display", "children", allow_duplicate=True),
    Output("datetime-slider", "value", allow_duplicate=True),
    Output("figure", "figure", allow_duplicate=True),

    Input("backward-button", "n_clicks"),
    State("datetime-slider", "disabled"),
    State("datetime-slider", "value"),
    State("datetime-window", "data"),
    State("figure", "figure"),

    prevent_initial_call=True,
)
def backward_button(n_clicks, player_disabled, slider_val, window_val, fig):
    slider_val = datetime.datetime.fromtimestamp(slider_val)
    if player_disabled:
        return datetime_display(slider_val, window_val), slider_val.timestamp(), fig
    slider_val -= datetime.timedelta(hours=1)
    slider_val = max(slider_val, min_date)
    dff = filter_df(df, slider_val, window_val)
    return datetime_display(slider_val, window_val), slider_val.timestamp(), build_map(dff)


@app.callback(
    Output("datetime-display", "children", allow_duplicate=True),
    Output("figure", "figure", allow_duplicate=True),

    Input("datetime-slider", "value"),
    State("datetime-window", "data"),
    State("home-button-flag", "data"),

    prevent_initial_call=True,
)
def datetime_slider(slider_val, window_val, home_button_flag):
    if home_button_flag:
        return "", build_map(df)
    slider_val = datetime.datetime.fromtimestamp(slider_val)
    dff = filter_df(df, slider_val, window_val)
    return datetime_display(slider_val, window_val), build_map(dff)


@app.callback(
    Output("datetime-display", "children", allow_duplicate=True),
    Output("datetime-slider", "value", allow_duplicate=True),
    Output("figure", "figure", allow_duplicate=True),

    Input("datetime-ticker", "n_intervals"),
    State("datetime-slider", "value"),
    State("datetime-window", "data"),

    prevent_initial_call=True,
)
def datetime_ticker(n_intervals, slider_val, window_val):
    slider_val = datetime.datetime.fromtimestamp(slider_val)
    slider_val += datetime.timedelta(hours=1)
    slider_val = min(slider_val, max_date)
    dff = filter_df(df, slider_val, window_val)
    return datetime_display(slider_val, window_val), slider_val.timestamp(), build_map(dff)


@app.callback(
    Output("datetime-display", "children", allow_duplicate=True),
    Output("datetime-slider", "value", allow_duplicate=True),
    Output("figure", "figure", allow_duplicate=True),

    Input("forward-button", "n_clicks"),
    State("datetime-slider", "disabled"),
    State("datetime-slider", "value"),
    State("datetime-window", "data"),
    State("figure", "figure"),

    prevent_initial_call=True,
)
def forward_button(n_clicks, player_disabled, slider_val, window_val, fig):
    slider_val = datetime.datetime.fromtimestamp(slider_val)
    if player_disabled:
        return datetime_display(slider_val, window_val), slider_val.timestamp()
    slider_val += datetime.timedelta(hours=1)
    slider_val = min(slider_val, max_date)
    dff = filter_df(df, slider_val, window_val)
    return datetime_display(slider_val, window_val), slider_val.timestamp(), build_map(dff)


@app.callback(
    Output("datetime-display", "children", allow_duplicate=True),
    Output("datetime-window", "data", allow_duplicate=True),

    Input("plus-button", "n_clicks"),
    State("datetime-slider", "disabled"),
    State("datetime-slider", "value"),
    State("datetime-window", "data"),

    prevent_initial_call=True,
)
def plus_button(n_clicks, player_disabled, slider_val, window_val):
    slider_val = datetime.datetime.fromtimestamp(slider_val)
    if not player_disabled:
        window_val += 1
    return datetime_display(slider_val, window_val), window_val


@app.callback(
    Output("datetime-display", "children", allow_duplicate=True),
    Output("datetime-window", "data", allow_duplicate=True),

    Input("minus-button", "n_clicks"),
    State("datetime-slider", "disabled"),
    State("datetime-slider", "value"),
    State("datetime-window", "data"),

    prevent_initial_call=True,
)
def minus_button(n_clicks, player_disabled, slider_val, window_val):
    slider_val = datetime.datetime.fromtimestamp(slider_val)
    if not player_disabled and window_val > 1:
        window_val -= 1
    return datetime_display(slider_val, window_val), window_val

In [11]:
app.run_server(port=8050, debug=True, use_reloader=False)

Dash is running on http://127.0.0.1:8050/

Dash app running on http://127.0.0.1:8050/


In [12]:
# webbrowser.open('http://127.0.0.1:8050/')
# app.run_server(port=8050, debug=True, use_reloader=False)
# time.sleep(5)
# app.run_server(port=8050, debug=True, use_reloader=False)